### The following code establishes connection with snowflake using snowflake.connector and sqlalchemy.
The code then:

1. executes sql code in python
2. loops through the sql code and outputs results 
3. pushes the final dataframe to snowflake as a table

In [ ]:
import snowflake.connector
from snowflake.connector.pandas_tools import write_pandas
from snowflake.connector.pandas_tools import pd_writer
from sqlalchemy import create_engine
from snowflake.sqlalchemy import URL
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as scs
from datetime import date, datetime
from time import strftime

In [ ]:
#connection
DATABASE = 'database' #update for your own account
schema =  'schema' #update for your own account
username = 'user' #update for your own account
account = 'account' #update for your own account
 
kwargs = {"authenticator":  "externalbrowser"} #options

connection = snowflake.connector.connect(
    user       = username,
    password   = "",
    account    = account, **kwargs
    )

#specify schema
cur = connection.cursor()
     
# define schema
cur.execute(f'USE {DATABASE}.{schema}')

# engine
engine = create_engine(URL(
account = 'account', #update for your own account
user = 'user', #update for your own account
database = 'database', #update for your own account
schema = 'schema', #update for your own account
authenticator='externalbrowser'))         

1. execute a sql code in python

In [ ]:
# Use connection to execute a query
query = f"""
SELECT *
    FROM TABLE 
    where entry_created_at::date < '2023-1-1'
"""

# Use cursor to execute a command
cur.execute(f'USE DATABASE') #update for your own account

# import to dataframe
df_prior = pd.read_sql(query, connection)

In [ ]:
# Alternatively, you can read from a text file that includes sql code
# open text file in read mode
text_file = open("./test.sql", "r")

# read whole file to a string
sql = text_file.read()

# close file
text_file.close()

# execute query
for chunk in sql.split(';'):
    print(chunk,'\n'*2)
    conn.execute(chunk)

# import to dataframe
df = pd.read_sql('select * from analytics.cte_pt_elig', con = connection)

2. loops through the sql code and outputs results

In [ ]:
# Dictionary to store dataframes
dataframe_collection = {}

# Create a list of simulation dataset dates to merge to the baseline - as of 2023-10, there should be 9
simulation_dates = ['2023_01_26', '2023_02_28', '2023_03_28', '2023_04_26', '2023_05_30', '2023_06_08', '2023_07_28', '2023_08_24', '2023_09_28']

for variable in simulation_dates:
    query_to_execute = query_template.format(variable=variable)
    dataframe_collection[f"df_sim_{variable}"] = pd.read_sql(query_to_execute, connection)
    dataframe_collection[f"df_sim_{variable}"].columns = dataframe_collection[f"df_sim_{variable}"].columns.str.lower()
    dataframe_collection[f"df_sim_{variable}"]['id_column'] = dataframe_collection[f"df_sim_{variable}"]['id_column'].astype(np.int64)
    dataframe_collection[f"df_sim_{variable}"]['risk_score'] = dataframe_collection[f"df_sim_{variable}"]['risk_score'].astype(float)
    dataframe_collection[f"df_sim_{variable}"]['metric_value'] = dataframe_collection[f"df_sim_{variable}"]['metric_value'].astype(float)

for threshold_value in threshold_list:
    df_simulation = df_simulation.assign(target_group = np.where(df_simulation['risk_metric_product'] > np.nanpercentile(df_simulation['risk_metric_product'], threshold_value, method='median_unbiased'), 1, 0))
    data_subset = df_simulation[df_simulation["target_group"] == 1]
    df_previous.columns = df_previous.columns.str.lower()
    df_previous['id_column'] = df_previous['id_column'].astype(np.int64)
    data_subset['id_column'] = data_subset['id_column'].astype(np.int64)

    merged_data = pd.merge(df_previous, data_subset, on='id_column', how='outer', indicator=True)
    new_entries = merged_data[(merged_data['_merge'] == 'right_only')].drop(columns=['_merge'])
    base_df = new_entries.copy()
    base_df['id_column'] = base_df['id_column'].astype(np.int64)
    current_percentage = 100 - threshold_value
    df_keys = [f"df_sim_{date}_threshold_{threshold_value}" for date in simulation_dates]

    for variable in simulation_dates:
        original_key = f"df_sim_{variable}"
        threshold_key = f"df_sim_{variable}_threshold_{threshold_value}"
        df_with_threshold = dataframe_collection[original_key].copy()
        df_with_threshold = df_with_threshold.assign(risk_metric_product=df_with_threshold['risk_score'] * df_with_threshold['metric_value'])
        df_with_threshold = df_with_threshold.assign(target_group=np.where(df_with_threshold['risk_metric_product'] > np.nanpercentile(df_with_threshold['risk_metric_product'], threshold_value, method='median_unbiased'), 1, 0))
        df_with_threshold = df_with_threshold[df_with_threshold['target_group'] == 1]
        dataframe_collection[threshold_key] = df_with_threshold
        print(f"Number of rows in {threshold_key}: {df_with_threshold.shape[0]}")

    if 1 <= 9 <= len(df_keys):
        for key in df_keys[:9]:         
            df_to_add = dataframe_collection[key]
            base_df.update(df_to_add)
            additional_rows = df_to_add[~df_to_add['id_column'].isin(base_df['id_column'])]
            base_df = pd.concat([base_df, additional_rows], ignore_index=True)
    else:
        print("Number out of range!")
    
    final_df = base_df.copy()
    df_target_group = pd.concat([final_df, pd.get_dummies(final_df['age_group'], prefix='age_group')], axis=1)
    boolean_columns = ['age_group_1', 'age_group_2', 'age_group_3', 'age_group_4', 'age_group_5', 'age_group_6'] 
    for col in boolean_columns:
        df_target_group[col] = df_target_group[col].astype(int)
    df_summary = df_target_group[['risk_score','metric_value']].describe(include='all')
    transposed_summary = df_summary.T[['mean']].round(3)
    transposed_summary.loc['total_rows'] = len(final_df)
    transposed_summary.loc['data_label'] = f"""{simulation_date}: Simulation {current_percentage}%"""
    ordered_index = ['total_rows', 'data_label'] + [col for col in transposed_summary.index if col not in ['total_rows', 'data_label']]
    transposed_summary = transposed_summary.reindex(ordered_index)
    thresholds.append(transposed_summary)


3. write the table back to snowflake

In [ ]:
df_target_uppercase = new_entries.copy()  # This creates a new copy of your original DataFrame
df_target_uppercase.columns = df_target_uppercase.columns.str.upper()
table_name = f'TARGET_DATA_{current_percentage}'

get_ipython().run_cell_magic('time', '', f"""
from snowflake.connector.pandas_tools import write_pandas

# use database
cur.execute(f'USE DATA_WAREHOUSE')

rows, columns = df_target_uppercase.shape
print(f'Uploading {rows:,} rows and {columns:,} columns to data_warehouse.analytics.{table_name}')

output = write_pandas(
            conn       = connection,
            df         = df_target_uppercase,
            table_name = table_name,
            database   = 'database',
            schema     = 'schema',
            quote_identifiers = False,
            auto_create_table = True,
            overwrite = True
            ) 
print(output)
""")
